In [ ]:
from google.colab import drive
import os  

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pydotplus

In [ ]:
#呼叫雲端硬碟

drive.mount("/content/drive")

In [ ]:
#呼叫雲端資料夾_檔案_1415完整資料
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data1415.csv')
df.head(20)

In [ ]:
#對site與season編碼

from sklearn.preprocessing import OneHotEncoder

data_dum = pd.get_dummies(df)
pd.DataFrame(data_dum)

In [ ]:
#編碼後將nf放到最後面

data_dum_nf = data_dum.pop("nf") 
data_dum["nf"] = data_dum_nf
print(data_dum.head())


In [ ]:
#編碼後的X與y

X = data_dum.iloc[:,:46]
y = data_dum.iloc[:,46]

In [ ]:
#呼叫套件
import numpy  #數字運算
from sklearn.tree import DecisionTreeRegressor  # 決策樹
from sklearn.preprocessing import LabelEncoder  # 將string 替換成 int
from sklearn.model_selection import train_test_split # 將資料集劃分
#from sklearn.externals import joblib  #將 model 儲存下來
from sklearn.metrics import accuracy_score #測試predict 資料正確
import pandas as pd  # 讀取 csv file



from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import roc_curve, auc

from sklearn import tree
from sklearn import datasets
import pydotplus
from sklearn.model_selection import train_test_split


import graphviz

from sklearn.tree import export_graphviz 
from IPython.display import Image
import pydotplus

In [ ]:
# LR用Grid search找最佳參數_編碼  重要特徵 嵌套
from sklearn.linear_model import LogisticRegression
import numpy as np


#正規化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

grid=[{'penalty':['l1'], 'C':[0.001, 0.01, 0.1, 1, 10], 'solver':['liblinear']},{'penalty':['l2'], 'C':[0.001, 0.01, 0.1, 1, 10]}] 
logreg = LogisticRegression()
clf = GridSearchCV(logreg, grid, scoring='roc_auc', n_jobs=-1) #
RSKF = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=87)


#欄位名稱用
Z = data_dum.iloc[:,:46]


acc = []
mcc = []
auroc = []
sen = []
spe = []
features = []





for train_index, test_index in RSKF.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Class count
    count_class_0, count_class_1 = y_train.value_counts()
    # Divide by class
    X_train_class_0 = X_train[y_train == 0]
    X_train_class_1 = X_train[y_train == 1]
    X_train_class_0_under = X_train_class_0.sample(count_class_1)
    X_train_under = pd.concat([X_train_class_0_under, X_train_class_1], axis=0)
    #Final X_train, y_train
    X_train_f= X_train_under
    y_train_f= y_train[X_train_under.index]



    #正規化
    X_train_f = pd.DataFrame(scaler.fit_transform(X_train_f))
    X_test = pd.DataFrame(scaler.transform(X_test)) 
    
    clf.fit(X_train_f, y_train_f)
    print("tuned hpyerparameters :(best parameters) ",clf.best_params_)
    
    print("Grid scores on development set:")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))



    y_pred = clf.best_estimator_.predict(X_test)
    acc.append(clf.best_estimator_.score(X_test, y_test))


    
    mcc.append(matthews_corrcoef(y_test, y_pred))
    fpr, tpr, thresholds = roc_curve(y_test, clf.best_estimator_.predict_proba(X_test)[:,1])
    auroc.append(auc(fpr, tpr))
    C_matrix = confusion_matrix(y_test, y_pred)
    sen.append(C_matrix[1,1]/(C_matrix[1,1]+C_matrix[1,0]))
    spe.append(C_matrix[0,0]/(C_matrix[0,0]+C_matrix[0,1]))

    coefs = clf.best_estimator_.coef_[0]


    feature_importances = pd.DataFrame({"Feature":Z.columns,"Coefficients":coefs})
    ranking = feature_importances.sort_values('Coefficients', ascending=False)
    print(ranking)

    features.append(coefs)


tuned hpyerparameters :(best parameters)  {'C': 0.1, 'penalty': 'l2'}
Grid scores on development set:
0.500 (+/-0.000) for {'C': 0.001, 'penalty': 'l1', 'solver': 'liblinear'}
0.500 (+/-0.000) for {'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}
0.888 (+/-0.061) for {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
0.882 (+/-0.053) for {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
0.867 (+/-0.057) for {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
0.880 (+/-0.095) for {'C': 0.001, 'penalty': 'l2'}
0.889 (+/-0.078) for {'C': 0.01, 'penalty': 'l2'}
0.894 (+/-0.064) for {'C': 0.1, 'penalty': 'l2'}
0.880 (+/-0.052) for {'C': 1, 'penalty': 'l2'}
0.870 (+/-0.055) for {'C': 10, 'penalty': 'l2'}
     Feature  Coefficients
38      band      0.764074
32       crp      0.762490
12   Hbullae      0.471473
28       wbc      0.426817
23      alco      0.403592
8     animal      0.372187
5       hypo      0.358542
7        sea      0.255058
24       htn      0.252808
18      crep      0.232

In [ ]:

#平均每一個特徵的權重


'''

F0 = (features[0][0] + features[1][0] + features[2][0] +features[3][0] +features[4][0]+features[5][0]+features[6][0]+features[7][0]+features[8][0]+features[9][0]+features[10][0] + features[11][0] + features[12][0] +features[13][0] +features[14][0]+features[15][0]+features[16][0]+features[17][0]+features[18][0]+features[19][0]+features[20][0] + features[21][0] + features[22][0] +features[23][0] +features[24][0]+features[25][0]+features[26][0]+features[27][0]+features[28][0]+features[29][0]+features[30][0] + features[31][0] + features[32][0] +features[33][0] +features[34][0]+features[35][0]+features[36][0]+features[37][0]+features[38][0]+features[39][0]+features[40][0] + features[41][0] + features[42][0] +features[43][0] +features[44][0]+features[45][0]+features[46][0]+features[47][0]+features[48][0]+features[49][0]) / 50
F1 = (features[0][1] + features[1][1] + features[2][1] +features[3][1] +features[4][1]+features[5][1]+features[6][1]+features[7][1]+features[8][1]+features[9][1]+features[10][1] + features[11][1] + features[12][1] +features[13][1] +features[14][1]+features[15][1]+features[16][1]+features[17][1]+features[18][1]+features[19][1]+features[20][1] + features[21][1] + features[22][1] +features[23][1] +features[24][1]+features[25][1]+features[26][1]+features[27][1]+features[28][1]+features[29][1]+features[30][1] + features[31][1] + features[32][1] +features[33][1] +features[34][1]+features[35][1]+features[36][1]+features[37][1]+features[38][1]+features[39][1]+features[40][1] + features[41][1] + features[42][1] +features[43][1] +features[44][1]+features[45][1]+features[46][1]+features[47][1]+features[48][1]+features[49][1]) / 50
F2 = (features[0][2] + features[1][2] + features[2][2] +features[3][2] +features[4][2]+features[5][2]+features[6][2]+features[7][2]+features[8][2]+features[9][2]+features[10][2] + features[11][2] + features[12][2] +features[13][2] +features[14][2]+features[15][2]+features[16][2]+features[17][2]+features[18][2]+features[19][2]+features[20][2] + features[21][2] + features[22][2] +features[23][2] +features[24][2]+features[25][2]+features[26][2]+features[27][2]+features[28][2]+features[29][2]+features[30][2] + features[31][2] + features[32][2] +features[33][2] +features[34][2]+features[35][2]+features[36][2]+features[37][2]+features[38][2]+features[39][2]+features[40][2] + features[41][2] + features[42][2] +features[43][2] +features[44][2]+features[45][2]+features[46][2]+features[47][2]+features[48][2]+features[49][2]) / 50
F3 = (features[0][3] + features[1][3] + features[2][3] +features[3][3] +features[4][3]+features[5][3]+features[6][3]+features[7][3]+features[8][3]+features[9][3]+features[10][3] + features[11][3] + features[12][3] +features[13][3] +features[14][3]+features[15][3]+features[16][3]+features[17][3]+features[18][3]+features[19][3]+features[20][3] + features[21][3] + features[22][3] +features[23][3] +features[24][3]+features[25][3]+features[26][3]+features[27][3]+features[28][3]+features[29][3]+features[30][3] + features[31][3] + features[32][3] +features[33][3] +features[34][3]+features[35][3]+features[36][3]+features[37][3]+features[38][3]+features[39][3]+features[40][3] + features[41][3] + features[42][3] +features[43][3] +features[44][3]+features[45][3]+features[46][3]+features[47][3]+features[48][3]+features[49][3]) / 50
F4 = (features[0][4] + features[1][4] + features[2][4] +features[3][4] +features[4][4]+features[5][4]+features[6][4]+features[7][4]+features[8][4]+features[9][4]+features[10][4] + features[11][4] + features[12][4] +features[13][4] +features[14][4]+features[15][4]+features[16][4]+features[17][4]+features[18][4]+features[19][4]+features[20][4] + features[21][4] + features[22][4] +features[23][4] +features[24][4]+features[25][4]+features[26][4]+features[27][4]+features[28][4]+features[29][4]+features[30][4] + features[31][4] + features[32][4] +features[33][4] +features[34][4]+features[35][4]+features[36][4]+features[37][4]+features[38][4]+features[39][4]+features[40][4] + features[41][4] + features[42][4] +features[43][4] +features[44][4]+features[45][4]+features[46][4]+features[47][4]+features[48][4]+features[49][4]) / 50
F5 = (features[0][5] + features[1][5] + features[2][5] +features[3][5] +features[4][5]+features[5][5]+features[6][5]+features[7][5]+features[8][5]+features[9][5]+features[10][5] + features[11][5] + features[12][5] +features[13][5] +features[14][5]+features[15][5]+features[16][5]+features[17][5]+features[18][5]+features[19][5]+features[20][5] + features[21][5] + features[22][5] +features[23][5] +features[24][5]+features[25][5]+features[26][5]+features[27][5]+features[28][5]+features[29][5]+features[30][5] + features[31][5] + features[32][5] +features[33][5] +features[34][5]+features[35][5]+features[36][5]+features[37][5]+features[38][5]+features[39][5]+features[40][5] + features[41][5] + features[42][5] +features[43][5] +features[44][5]+features[45][5]+features[46][5]+features[47][5]+features[48][5]+features[49][5]) / 50
F6 = (features[0][6] + features[1][6] + features[2][6] +features[3][6] +features[4][6]+features[5][6]+features[6][6]+features[7][6]+features[8][6]+features[9][6]+features[10][6] + features[11][6] + features[12][6] +features[13][6] +features[14][6]+features[15][6]+features[16][6]+features[17][6]+features[18][6]+features[19][6]+features[20][6] + features[21][6] + features[22][6] +features[23][6] +features[24][6]+features[25][6]+features[26][6]+features[27][6]+features[28][6]+features[29][6]+features[30][6] + features[31][6] + features[32][6] +features[33][6] +features[34][6]+features[35][6]+features[36][6]+features[37][6]+features[38][6]+features[39][6]+features[40][6] + features[41][6] + features[42][6] +features[43][6] +features[44][6]+features[45][6]+features[46][6]+features[47][6]+features[48][6]+features[49][6]) / 50
F7 = (features[0][7] + features[1][7] + features[2][7] +features[3][7] +features[4][7]+features[5][7]+features[6][7]+features[7][7]+features[8][7]+features[9][7]+features[10][7] + features[11][7] + features[12][7] +features[13][7] +features[14][7]+features[15][7]+features[16][7]+features[17][7]+features[18][7]+features[19][7]+features[20][7] + features[21][7] + features[22][7] +features[23][7] +features[24][7]+features[25][7]+features[26][7]+features[27][7]+features[28][7]+features[29][7]+features[30][7] + features[31][7] + features[32][7] +features[33][7] +features[34][7]+features[35][7]+features[36][7]+features[37][7]+features[38][7]+features[39][7]+features[40][7] + features[41][7] + features[42][7] +features[43][7] +features[44][7]+features[45][7]+features[46][7]+features[47][7]+features[48][7]+features[49][7]) / 50
F8 = (features[0][8] + features[1][8] + features[2][8] +features[3][8] +features[4][8]+features[5][8]+features[6][8]+features[7][8]+features[8][8]+features[9][8]+features[10][8] + features[11][8] + features[12][8] +features[13][8] +features[14][8]+features[15][8]+features[16][8]+features[17][8]+features[18][8]+features[19][8]+features[20][8] + features[21][8] + features[22][8] +features[23][8] +features[24][8]+features[25][8]+features[26][8]+features[27][8]+features[28][8]+features[29][8]+features[30][8] + features[31][8] + features[32][8] +features[33][8] +features[34][8]+features[35][8]+features[36][8]+features[37][8]+features[38][8]+features[39][8]+features[40][8] + features[41][8] + features[42][8] +features[43][8] +features[44][8]+features[45][8]+features[46][8]+features[47][8]+features[48][8]+features[49][8]) / 50
F9 = (features[0][9] + features[1][9] + features[2][9] +features[3][9] +features[4][9]+features[5][9]+features[6][9]+features[7][9]+features[8][9]+features[9][9]+features[10][9] + features[11][9] + features[12][9] +features[13][9] +features[14][9]+features[15][9]+features[16][9]+features[17][9]+features[18][9]+features[19][9]+features[20][9] + features[21][9] + features[22][9] +features[23][9] +features[24][9]+features[25][9]+features[26][9]+features[27][9]+features[28][9]+features[29][9]+features[30][9] + features[31][9] + features[32][9] +features[33][9] +features[34][9]+features[35][9]+features[36][9]+features[37][9]+features[38][9]+features[39][9]+features[40][9] + features[41][9] + features[42][9] +features[43][9] +features[44][9]+features[45][9]+features[46][9]+features[47][9]+features[48][9]+features[49][9]) / 50

F10 = (features[0][10] + features[1][10] + features[2][10] +features[3][10] +features[4][10]+features[5][10]+features[6][10]+features[7][10]+features[8][10]+features[9][10]+features[10][10] + features[11][10] + features[12][10] +features[13][10] +features[14][10]+features[15][10]+features[16][10]+features[17][10]+features[18][10]+features[19][10]+features[20][10] + features[21][10] + features[22][10] +features[23][10] +features[24][10]+features[25][10]+features[26][10]+features[27][10]+features[28][10]+features[29][10]+features[30][10] + features[31][10] + features[32][10] +features[33][10] +features[34][10]+features[35][10]+features[36][10]+features[37][10]+features[38][10]+features[39][10]+features[40][10] + features[41][10] + features[42][10] +features[43][10] +features[44][10]+features[45][10]+features[46][10]+features[47][10]+features[48][10]+features[49][10]) / 50
F11 = (features[0][11] + features[1][11] + features[2][11] +features[3][11] +features[4][11]+features[5][11]+features[6][11]+features[7][11]+features[8][11]+features[9][11]+features[10][11] + features[11][11] + features[12][11] +features[13][11] +features[14][11]+features[15][11]+features[16][11]+features[17][11]+features[18][11]+features[19][11]+features[20][11] + features[21][11] + features[22][11] +features[23][11] +features[24][11]+features[25][11]+features[26][11]+features[27][11]+features[28][11]+features[29][11]+features[30][11] + features[31][11] + features[32][11] +features[33][11] +features[34][11]+features[35][11]+features[36][11]+features[37][11]+features[38][11]+features[39][11]+features[40][11] + features[41][11] + features[42][11] +features[43][11] +features[44][11]+features[45][11]+features[46][11]+features[47][11]+features[48][11]+features[49][11]) / 50
F12 = (features[0][12] + features[1][12] + features[2][12] +features[3][12] +features[4][12]+features[5][12]+features[6][12]+features[7][12]+features[8][12]+features[9][12]+features[10][12] + features[11][12] + features[12][12] +features[13][12] +features[14][12]+features[15][12]+features[16][12]+features[17][12]+features[18][12]+features[19][12]+features[20][12] + features[21][12] + features[22][12] +features[23][12] +features[24][12]+features[25][12]+features[26][12]+features[27][12]+features[28][12]+features[29][12]+features[30][12] + features[31][12] + features[32][12] +features[33][12] +features[34][12]+features[35][12]+features[36][12]+features[37][12]+features[38][12]+features[39][12]+features[40][12] + features[41][12] + features[42][12] +features[43][12] +features[44][12]+features[45][12]+features[46][12]+features[47][12]+features[48][12]+features[49][12]) / 50
F13 = (features[0][13] + features[1][13] + features[2][13] +features[3][13] +features[4][13]+features[5][13]+features[6][13]+features[7][13]+features[8][13]+features[9][13]+features[10][13] + features[11][13] + features[12][13] +features[13][13] +features[14][13]+features[15][13]+features[16][13]+features[17][13]+features[18][13]+features[19][13]+features[20][13] + features[21][13] + features[22][13] +features[23][13] +features[24][13]+features[25][13]+features[26][13]+features[27][13]+features[28][13]+features[29][13]+features[30][13] + features[31][13] + features[32][13] +features[33][13] +features[34][13]+features[35][13]+features[36][13]+features[37][13]+features[38][13]+features[39][13]+features[40][13] + features[41][13] + features[42][13] +features[43][13] +features[44][13]+features[45][13]+features[46][13]+features[47][13]+features[48][13]+features[49][13]) / 50
F14 = (features[0][14] + features[1][14] + features[2][14] +features[3][14] +features[4][14]+features[5][14]+features[6][14]+features[7][14]+features[8][14]+features[9][14]+features[10][14] + features[11][14] + features[12][14] +features[13][14] +features[14][14]+features[15][14]+features[16][14]+features[17][14]+features[18][14]+features[19][14]+features[20][14] + features[21][14] + features[22][14] +features[23][14] +features[24][14]+features[25][14]+features[26][14]+features[27][14]+features[28][14]+features[29][14]+features[30][14] + features[31][14] + features[32][14] +features[33][14] +features[34][14]+features[35][14]+features[36][14]+features[37][14]+features[38][14]+features[39][14]+features[40][14] + features[41][14] + features[42][14] +features[43][14] +features[44][14]+features[45][14]+features[46][14]+features[47][14]+features[48][14]+features[49][14]) / 50
F15 = (features[0][15] + features[1][15] + features[2][15] +features[3][15] +features[4][15]+features[5][15]+features[6][15]+features[7][15]+features[8][15]+features[9][15]+features[10][15] + features[11][15] + features[12][15] +features[13][15] +features[14][15]+features[15][15]+features[16][15]+features[17][15]+features[18][15]+features[19][15]+features[20][15] + features[21][15] + features[22][15] +features[23][15] +features[24][15]+features[25][15]+features[26][15]+features[27][15]+features[28][15]+features[29][15]+features[30][15] + features[31][15] + features[32][15] +features[33][15] +features[34][15]+features[35][15]+features[36][15]+features[37][15]+features[38][15]+features[39][15]+features[40][15] + features[41][15] + features[42][15] +features[43][15] +features[44][15]+features[45][15]+features[46][15]+features[47][15]+features[48][15]+features[49][15]) / 50
F16 = (features[0][16] + features[1][16] + features[2][16] +features[3][16] +features[4][16]+features[5][16]+features[6][16]+features[7][16]+features[8][16]+features[9][16]+features[10][16] + features[11][16] + features[12][16] +features[13][16] +features[14][16]+features[15][16]+features[16][16]+features[17][16]+features[18][16]+features[19][16]+features[20][16] + features[21][16] + features[22][16] +features[23][16] +features[24][16]+features[25][16]+features[26][16]+features[27][16]+features[28][16]+features[29][16]+features[30][16] + features[31][16] + features[32][16] +features[33][16] +features[34][16]+features[35][16]+features[36][16]+features[37][16]+features[38][16]+features[39][16]+features[40][16] + features[41][16] + features[42][16] +features[43][16] +features[44][16]+features[45][16]+features[46][16]+features[47][16]+features[48][16]+features[49][16]) / 50
F17 = (features[0][17] + features[1][17] + features[2][17] +features[3][17] +features[4][17]+features[5][17]+features[6][17]+features[7][17]+features[8][17]+features[9][17]+features[10][17] + features[11][17] + features[12][17] +features[13][17] +features[14][17]+features[15][17]+features[16][17]+features[17][17]+features[18][17]+features[19][17]+features[20][17] + features[21][17] + features[22][17] +features[23][17] +features[24][17]+features[25][17]+features[26][17]+features[27][17]+features[28][17]+features[29][17]+features[30][17] + features[31][17] + features[32][17] +features[33][17] +features[34][17]+features[35][17]+features[36][17]+features[37][17]+features[38][17]+features[39][17]+features[40][17] + features[41][17] + features[42][17] +features[43][17] +features[44][17]+features[45][17]+features[46][17]+features[47][17]+features[48][17]+features[49][17]) / 50
F18 = (features[0][18] + features[1][18] + features[2][18] +features[3][18] +features[4][18]+features[5][18]+features[6][18]+features[7][18]+features[8][18]+features[9][18]+features[10][18] + features[11][18] + features[12][18] +features[13][18] +features[14][18]+features[15][18]+features[16][18]+features[17][18]+features[18][18]+features[19][18]+features[20][18] + features[21][18] + features[22][18] +features[23][18] +features[24][18]+features[25][18]+features[26][18]+features[27][18]+features[28][18]+features[29][18]+features[30][18] + features[31][18] + features[32][18] +features[33][18] +features[34][18]+features[35][18]+features[36][18]+features[37][18]+features[38][18]+features[39][18]+features[40][18] + features[41][18] + features[42][18] +features[43][18] +features[44][18]+features[45][18]+features[46][18]+features[47][18]+features[48][18]+features[49][18]) / 50
F19 = (features[0][19] + features[1][19] + features[2][19] +features[3][19] +features[4][19]+features[5][19]+features[6][19]+features[7][19]+features[8][19]+features[9][19]+features[10][19] + features[11][19] + features[12][19] +features[13][19] +features[14][19]+features[15][19]+features[16][19]+features[17][19]+features[18][19]+features[19][19]+features[20][19] + features[21][19] + features[22][19] +features[23][19] +features[24][19]+features[25][19]+features[26][19]+features[27][19]+features[28][19]+features[29][19]+features[30][19] + features[31][19] + features[32][19] +features[33][19] +features[34][19]+features[35][19]+features[36][19]+features[37][19]+features[38][19]+features[39][19]+features[40][19] + features[41][19] + features[42][19] +features[43][19] +features[44][19]+features[45][19]+features[46][19]+features[47][19]+features[48][19]+features[49][19]) / 50
F20 = (features[0][20] + features[1][20] + features[2][20] +features[3][20] +features[4][20]+features[5][20]+features[6][20]+features[7][20]+features[8][20]+features[9][20]+features[10][20] + features[11][20] + features[12][20] +features[13][20] +features[14][20]+features[15][20]+features[16][20]+features[17][20]+features[18][20]+features[19][20]+features[20][20] + features[21][20] + features[22][20] +features[23][20] +features[24][20]+features[25][20]+features[26][20]+features[27][20]+features[28][20]+features[29][20]+features[30][20] + features[31][20] + features[32][20] +features[33][20] +features[34][20]+features[35][20]+features[36][20]+features[37][20]+features[38][20]+features[39][20]+features[40][20] + features[41][20] + features[42][20] +features[43][20] +features[44][20]+features[45][20]+features[46][20]+features[47][20]+features[48][20]+features[49][20]) / 50
F21 = (features[0][21] + features[1][21] + features[2][21] +features[3][21] +features[4][21]+features[5][21]+features[6][21]+features[7][21]+features[8][21]+features[9][21]+features[10][21] + features[11][21] + features[12][21] +features[13][21] +features[14][21]+features[15][21]+features[16][21]+features[17][21]+features[18][21]+features[19][21]+features[20][21] + features[21][21] + features[22][21] +features[23][21] +features[24][21]+features[25][21]+features[26][21]+features[27][21]+features[28][21]+features[29][21]+features[30][21] + features[31][21] + features[32][21] +features[33][21] +features[34][21]+features[35][21]+features[36][21]+features[37][21]+features[38][21]+features[39][21]+features[40][21] + features[41][21] + features[42][21] +features[43][21] +features[44][21]+features[45][21]+features[46][21]+features[47][21]+features[48][21]+features[49][21]) / 50
F22 = (features[0][22] + features[1][22] + features[2][22] +features[3][22] +features[4][22]+features[5][22]+features[6][22]+features[7][22]+features[8][22]+features[9][22]+features[10][22] + features[11][22] + features[12][22] +features[13][22] +features[14][22]+features[15][22]+features[16][22]+features[17][22]+features[18][22]+features[19][22]+features[20][22] + features[21][22] + features[22][22] +features[23][22] +features[24][22]+features[25][22]+features[26][22]+features[27][22]+features[28][22]+features[29][22]+features[30][22] + features[31][22] + features[32][22] +features[33][22] +features[34][22]+features[35][22]+features[36][22]+features[37][22]+features[38][22]+features[39][22]+features[40][22] + features[41][22] + features[42][22] +features[43][22] +features[44][22]+features[45][22]+features[46][22]+features[47][22]+features[48][22]+features[49][22]) / 50
F23 = (features[0][23] + features[1][23] + features[2][23] +features[3][23] +features[4][23]+features[5][23]+features[6][23]+features[7][23]+features[8][23]+features[9][23]+features[10][23] + features[11][23] + features[12][23] +features[13][23] +features[14][23]+features[15][23]+features[16][23]+features[17][23]+features[18][23]+features[19][23]+features[20][23] + features[21][23] + features[22][23] +features[23][23] +features[24][23]+features[25][23]+features[26][23]+features[27][23]+features[28][23]+features[29][23]+features[30][23] + features[31][23] + features[32][23] +features[33][23] +features[34][23]+features[35][23]+features[36][23]+features[37][23]+features[38][23]+features[39][23]+features[40][23] + features[41][23] + features[42][23] +features[43][23] +features[44][23]+features[45][23]+features[46][23]+features[47][23]+features[48][23]+features[49][23]) / 50
F24 = (features[0][24] + features[1][24] + features[2][24] +features[3][24] +features[4][24]+features[5][24]+features[6][24]+features[7][24]+features[8][24]+features[9][24]+features[10][24] + features[11][24] + features[12][24] +features[13][24] +features[14][24]+features[15][24]+features[16][24]+features[17][24]+features[18][24]+features[19][24]+features[20][24] + features[21][24] + features[22][24] +features[23][24] +features[24][24]+features[25][24]+features[26][24]+features[27][24]+features[28][24]+features[29][24]+features[30][24] + features[31][24] + features[32][24] +features[33][24] +features[34][24]+features[35][24]+features[36][24]+features[37][24]+features[38][24]+features[39][24]+features[40][24] + features[41][24] + features[42][24] +features[43][24] +features[44][24]+features[45][24]+features[46][24]+features[47][24]+features[48][24]+features[49][24]) / 50
F25 = (features[0][25] + features[1][25] + features[2][25] +features[3][25] +features[4][25]+features[5][25]+features[6][25]+features[7][25]+features[8][25]+features[9][25]+features[10][25] + features[11][25] + features[12][25] +features[13][25] +features[14][25]+features[15][25]+features[16][25]+features[17][25]+features[18][25]+features[19][25]+features[20][25] + features[21][25] + features[22][25] +features[23][25] +features[24][25]+features[25][25]+features[26][25]+features[27][25]+features[28][25]+features[29][25]+features[30][25] + features[31][25] + features[32][25] +features[33][25] +features[34][25]+features[35][25]+features[36][25]+features[37][25]+features[38][25]+features[39][25]+features[40][25] + features[41][25] + features[42][25] +features[43][25] +features[44][25]+features[45][25]+features[46][25]+features[47][25]+features[48][25]+features[49][25]) / 50
F26 = (features[0][26] + features[1][26] + features[2][26] +features[3][26] +features[4][26]+features[5][26]+features[6][26]+features[7][26]+features[8][26]+features[9][26]+features[10][26] + features[11][26] + features[12][26] +features[13][26] +features[14][26]+features[15][26]+features[16][26]+features[17][26]+features[18][26]+features[19][26]+features[20][26] + features[21][26] + features[22][26] +features[23][26] +features[24][26]+features[25][26]+features[26][26]+features[27][26]+features[28][26]+features[29][26]+features[30][26] + features[31][26] + features[32][26] +features[33][26] +features[34][26]+features[35][26]+features[36][26]+features[37][26]+features[38][26]+features[39][26]+features[40][26] + features[41][26] + features[42][26] +features[43][26] +features[44][26]+features[45][26]+features[46][26]+features[47][26]+features[48][26]+features[49][26]) / 50
F27 = (features[0][27] + features[1][27] + features[2][27] +features[3][27] +features[4][27]+features[5][27]+features[6][27]+features[7][27]+features[8][27]+features[9][27]+features[10][27] + features[11][27] + features[12][27] +features[13][27] +features[14][27]+features[15][27]+features[16][27]+features[17][27]+features[18][27]+features[19][27]+features[20][27] + features[21][27] + features[22][27] +features[23][27] +features[24][27]+features[25][27]+features[26][27]+features[27][27]+features[28][27]+features[29][27]+features[30][27] + features[31][27] + features[32][27] +features[33][27] +features[34][27]+features[35][27]+features[36][27]+features[37][27]+features[38][27]+features[39][27]+features[40][27] + features[41][27] + features[42][27] +features[43][27] +features[44][27]+features[45][27]+features[46][27]+features[47][27]+features[48][27]+features[49][27]) / 50
F28 = (features[0][28] + features[1][28] + features[2][28] +features[3][28] +features[4][28]+features[5][28]+features[6][28]+features[7][28]+features[8][28]+features[9][28]+features[10][28] + features[11][28] + features[12][28] +features[13][28] +features[14][28]+features[15][28]+features[16][28]+features[17][28]+features[18][28]+features[19][28]+features[20][28] + features[21][28] + features[22][28] +features[23][28] +features[24][28]+features[25][28]+features[26][28]+features[27][28]+features[28][28]+features[29][28]+features[30][28] + features[31][28] + features[32][28] +features[33][28] +features[34][28]+features[35][28]+features[36][28]+features[37][28]+features[38][28]+features[39][28]+features[40][28] + features[41][28] + features[42][28] +features[43][28] +features[44][28]+features[45][28]+features[46][28]+features[47][28]+features[48][28]+features[49][28]) / 50
F29 = (features[0][29] + features[1][29] + features[2][29] +features[3][29] +features[4][29]+features[5][29]+features[6][29]+features[7][29]+features[8][29]+features[9][29]+features[10][29] + features[11][29] + features[12][29] +features[13][29] +features[14][29]+features[15][29]+features[16][29]+features[17][29]+features[18][29]+features[19][29]+features[20][29] + features[21][29] + features[22][29] +features[23][29] +features[24][29]+features[25][29]+features[26][29]+features[27][29]+features[28][29]+features[29][29]+features[30][29] + features[31][29] + features[32][29] +features[33][29] +features[34][29]+features[35][29]+features[36][29]+features[37][29]+features[38][29]+features[39][29]+features[40][29] + features[41][29] + features[42][29] +features[43][29] +features[44][29]+features[45][29]+features[46][29]+features[47][29]+features[48][29]+features[49][29]) / 50
F30 = (features[0][30] + features[1][30] + features[2][30] +features[3][30] +features[4][30]+features[5][30]+features[6][30]+features[7][30]+features[8][30]+features[9][30]+features[10][30] + features[11][30] + features[12][30] +features[13][30] +features[14][30]+features[15][30]+features[16][30]+features[17][30]+features[18][30]+features[19][30]+features[20][30] + features[21][30] + features[22][30] +features[23][30] +features[24][30]+features[25][30]+features[26][30]+features[27][30]+features[28][30]+features[29][30]+features[30][30] + features[31][30] + features[32][30] +features[33][30] +features[34][30]+features[35][30]+features[36][30]+features[37][30]+features[38][30]+features[39][30]+features[40][30] + features[41][30] + features[42][30] +features[43][30] +features[44][30]+features[45][30]+features[46][30]+features[47][30]+features[48][30]+features[49][30]) / 50
F31 = (features[0][31] + features[1][31] + features[2][31] +features[3][31] +features[4][31]+features[5][31]+features[6][31]+features[7][31]+features[8][31]+features[9][31]+features[10][31] + features[11][31] + features[12][31] +features[13][31] +features[14][31]+features[15][31]+features[16][31]+features[17][31]+features[18][31]+features[19][31]+features[20][31] + features[21][31] + features[22][31] +features[23][31] +features[24][31]+features[25][31]+features[26][31]+features[27][31]+features[28][31]+features[29][31]+features[30][31] + features[31][31] + features[32][31] +features[33][31] +features[34][31]+features[35][31]+features[36][31]+features[37][31]+features[38][31]+features[39][31]+features[40][31] + features[41][31] + features[42][31] +features[43][31] +features[44][31]+features[45][31]+features[46][31]+features[47][31]+features[48][31]+features[49][31]) / 50
F32 = (features[0][32] + features[1][32] + features[2][32] +features[3][32] +features[4][32]+features[5][32]+features[6][32]+features[7][32]+features[8][32]+features[9][32]+features[10][32] + features[11][32] + features[12][32] +features[13][32] +features[14][32]+features[15][32]+features[16][32]+features[17][32]+features[18][32]+features[19][32]+features[20][32] + features[21][32] + features[22][32] +features[23][32] +features[24][32]+features[25][32]+features[26][32]+features[27][32]+features[28][32]+features[29][32]+features[30][32] + features[31][32] + features[32][32] +features[33][32] +features[34][32]+features[35][32]+features[36][32]+features[37][32]+features[38][32]+features[39][32]+features[40][32] + features[41][32] + features[42][32] +features[43][32] +features[44][32]+features[45][32]+features[46][32]+features[47][32]+features[48][32]+features[49][32]) / 50
F33 = (features[0][33] + features[1][33] + features[2][33] +features[3][33] +features[4][33]+features[5][33]+features[6][33]+features[7][33]+features[8][33]+features[9][33]+features[10][33] + features[11][33] + features[12][33] +features[13][33] +features[14][33]+features[15][33]+features[16][33]+features[17][33]+features[18][33]+features[19][33]+features[20][33] + features[21][33] + features[22][33] +features[23][33] +features[24][33]+features[25][33]+features[26][33]+features[27][33]+features[28][33]+features[29][33]+features[30][33] + features[31][33] + features[32][33] +features[33][33] +features[34][33]+features[35][33]+features[36][33]+features[37][33]+features[38][33]+features[39][33]+features[40][33] + features[41][33] + features[42][33] +features[43][33] +features[44][33]+features[45][33]+features[46][33]+features[47][33]+features[48][33]+features[49][33]) / 50
F34 = (features[0][34] + features[1][34] + features[2][34] +features[3][34] +features[4][34]+features[5][34]+features[6][34]+features[7][34]+features[8][34]+features[9][34]+features[10][34] + features[11][34] + features[12][34] +features[13][34] +features[14][34]+features[15][34]+features[16][34]+features[17][34]+features[18][34]+features[19][34]+features[20][34] + features[21][34] + features[22][34] +features[23][34] +features[24][34]+features[25][34]+features[26][34]+features[27][34]+features[28][34]+features[29][34]+features[30][34] + features[31][34] + features[32][34] +features[33][34] +features[34][34]+features[35][34]+features[36][34]+features[37][34]+features[38][34]+features[39][34]+features[40][34] + features[41][34] + features[42][34] +features[43][34] +features[44][34]+features[45][34]+features[46][34]+features[47][34]+features[48][34]+features[49][34]) / 50
F35 = (features[0][35] + features[1][35] + features[2][35] +features[3][35] +features[4][35]+features[5][35]+features[6][35]+features[7][35]+features[8][35]+features[9][35]+features[10][35] + features[11][35] + features[12][35] +features[13][35] +features[14][35]+features[15][35]+features[16][35]+features[17][35]+features[18][35]+features[19][35]+features[20][35] + features[21][35] + features[22][35] +features[23][35] +features[24][35]+features[25][35]+features[26][35]+features[27][35]+features[28][35]+features[29][35]+features[30][35] + features[31][35] + features[32][35] +features[33][35] +features[34][35]+features[35][35]+features[36][35]+features[37][35]+features[38][35]+features[39][35]+features[40][35] + features[41][35] + features[42][35] +features[43][35] +features[44][35]+features[45][35]+features[46][35]+features[47][35]+features[48][35]+features[49][35]) / 50
F36 = (features[0][36] + features[1][36] + features[2][36] +features[3][36] +features[4][36]+features[5][36]+features[6][36]+features[7][36]+features[8][36]+features[9][36]+features[10][36] + features[11][36] + features[12][36] +features[13][36] +features[14][36]+features[15][36]+features[16][36]+features[17][36]+features[18][36]+features[19][36]+features[20][36] + features[21][36] + features[22][36] +features[23][36] +features[24][36]+features[25][36]+features[26][36]+features[27][36]+features[28][36]+features[29][36]+features[30][36] + features[31][36] + features[32][36] +features[33][36] +features[34][36]+features[35][36]+features[36][36]+features[37][36]+features[38][36]+features[39][36]+features[40][36] + features[41][36] + features[42][36] +features[43][36] +features[44][36]+features[45][36]+features[46][36]+features[47][36]+features[48][36]+features[49][36]) / 50
F37 = (features[0][37] + features[1][37] + features[2][37] +features[3][37] +features[4][37]+features[5][37]+features[6][37]+features[7][37]+features[8][37]+features[9][37]+features[10][37] + features[11][37] + features[12][37] +features[13][37] +features[14][37]+features[15][37]+features[16][37]+features[17][37]+features[18][37]+features[19][37]+features[20][37] + features[21][37] + features[22][37] +features[23][37] +features[24][37]+features[25][37]+features[26][37]+features[27][37]+features[28][37]+features[29][37]+features[30][37] + features[31][37] + features[32][37] +features[33][37] +features[34][37]+features[35][37]+features[36][37]+features[37][37]+features[38][37]+features[39][37]+features[40][37] + features[41][37] + features[42][37] +features[43][37] +features[44][37]+features[45][37]+features[46][37]+features[47][37]+features[48][37]+features[49][37]) / 50
F38 = (features[0][38] + features[1][38] + features[2][38] +features[3][38] +features[4][38]+features[5][38]+features[6][38]+features[7][38]+features[8][38]+features[9][38]+features[10][38] + features[11][38] + features[12][38] +features[13][38] +features[14][38]+features[15][38]+features[16][38]+features[17][38]+features[18][38]+features[19][38]+features[20][38] + features[21][38] + features[22][38] +features[23][38] +features[24][38]+features[25][38]+features[26][38]+features[27][38]+features[28][38]+features[29][38]+features[30][38] + features[31][38] + features[32][38] +features[33][38] +features[34][38]+features[35][38]+features[36][38]+features[37][38]+features[38][38]+features[39][38]+features[40][38] + features[41][38] + features[42][38] +features[43][38] +features[44][38]+features[45][38]+features[46][38]+features[47][38]+features[48][38]+features[49][38]) / 50
F39 = (features[0][39] + features[1][39] + features[2][39] +features[3][39] +features[4][39]+features[5][39]+features[6][39]+features[7][39]+features[8][39]+features[9][39]+features[10][39] + features[11][39] + features[12][39] +features[13][39] +features[14][39]+features[15][39]+features[16][39]+features[17][39]+features[18][39]+features[19][39]+features[20][39] + features[21][39] + features[22][39] +features[23][39] +features[24][39]+features[25][39]+features[26][39]+features[27][39]+features[28][39]+features[29][39]+features[30][39] + features[31][39] + features[32][39] +features[33][39] +features[34][39]+features[35][39]+features[36][39]+features[37][39]+features[38][39]+features[39][39]+features[40][39] + features[41][39] + features[42][39] +features[43][39] +features[44][39]+features[45][39]+features[46][39]+features[47][39]+features[48][39]+features[49][39]) / 50
F40 = (features[0][40] + features[1][40] + features[2][40] +features[3][40] +features[4][40]+features[5][40]+features[6][40]+features[7][40]+features[8][40]+features[9][40]+features[10][40] + features[11][40] + features[12][40] +features[13][40] +features[14][40]+features[15][40]+features[16][40]+features[17][40]+features[18][40]+features[19][40]+features[20][40] + features[21][40] + features[22][40] +features[23][40] +features[24][40]+features[25][40]+features[26][40]+features[27][40]+features[28][40]+features[29][40]+features[30][40] + features[31][40] + features[32][40] +features[33][40] +features[34][40]+features[35][40]+features[36][40]+features[37][40]+features[38][40]+features[39][40]+features[40][40] + features[41][40] + features[42][40] +features[43][40] +features[44][40]+features[45][40]+features[46][40]+features[47][40]+features[48][40]+features[49][40]) / 50
F41 = (features[0][41] + features[1][41] + features[2][41] +features[3][41] +features[4][41]+features[5][41]+features[6][41]+features[7][41]+features[8][41]+features[9][41]+features[10][41] + features[11][41] + features[12][41] +features[13][41] +features[14][41]+features[15][41]+features[16][41]+features[17][41]+features[18][41]+features[19][41]+features[20][41] + features[21][41] + features[22][41] +features[23][41] +features[24][41]+features[25][41]+features[26][41]+features[27][41]+features[28][41]+features[29][41]+features[30][41] + features[31][41] + features[32][41] +features[33][41] +features[34][41]+features[35][41]+features[36][41]+features[37][41]+features[38][41]+features[39][41]+features[40][41] + features[41][41] + features[42][41] +features[43][41] +features[44][41]+features[45][41]+features[46][41]+features[47][41]+features[48][41]+features[49][41]) / 50
F42 = (features[0][42] + features[1][42] + features[2][42] +features[3][42] +features[4][42]+features[5][42]+features[6][42]+features[7][42]+features[8][42]+features[9][42]+features[10][42] + features[11][42] + features[12][42] +features[13][42] +features[14][42]+features[15][42]+features[16][42]+features[17][42]+features[18][42]+features[19][42]+features[20][42] + features[21][42] + features[22][42] +features[23][42] +features[24][42]+features[25][42]+features[26][42]+features[27][42]+features[28][42]+features[29][42]+features[30][42] + features[31][42] + features[32][42] +features[33][42] +features[34][42]+features[35][42]+features[36][42]+features[37][42]+features[38][42]+features[39][42]+features[40][42] + features[41][42] + features[42][42] +features[43][42] +features[44][42]+features[45][42]+features[46][42]+features[47][42]+features[48][42]+features[49][42]) / 50
F43 = (features[0][43] + features[1][43] + features[2][43] +features[3][43] +features[4][43]+features[5][43]+features[6][43]+features[7][43]+features[8][43]+features[9][43]+features[10][43] + features[11][43] + features[12][43] +features[13][43] +features[14][43]+features[15][43]+features[16][43]+features[17][43]+features[18][43]+features[19][43]+features[20][43] + features[21][43] + features[22][43] +features[23][43] +features[24][43]+features[25][43]+features[26][43]+features[27][43]+features[28][43]+features[29][43]+features[30][43] + features[31][43] + features[32][43] +features[33][43] +features[34][43]+features[35][43]+features[36][43]+features[37][43]+features[38][43]+features[39][43]+features[40][43] + features[41][43] + features[42][43] +features[43][43] +features[44][43]+features[45][43]+features[46][43]+features[47][43]+features[48][43]+features[49][43]) / 50
F44 = (features[0][44] + features[1][44] + features[2][44] +features[3][44] +features[4][44]+features[5][44]+features[6][44]+features[7][44]+features[8][44]+features[9][44]+features[10][44] + features[11][44] + features[12][44] +features[13][44] +features[14][44]+features[15][44]+features[16][44]+features[17][44]+features[18][44]+features[19][44]+features[20][44] + features[21][44] + features[22][44] +features[23][44] +features[24][44]+features[25][44]+features[26][44]+features[27][44]+features[28][44]+features[29][44]+features[30][44] + features[31][44] + features[32][44] +features[33][44] +features[34][44]+features[35][44]+features[36][44]+features[37][44]+features[38][44]+features[39][44]+features[40][44] + features[41][44] + features[42][44] +features[43][44] +features[44][44]+features[45][44]+features[46][44]+features[47][44]+features[48][44]+features[49][44]) / 50
F45 = (features[0][45] + features[1][45] + features[2][45] +features[3][45] +features[4][45]+features[5][45]+features[6][45]+features[7][45]+features[8][45]+features[9][45]+features[10][45] + features[11][45] + features[12][45] +features[13][45] +features[14][45]+features[15][45]+features[16][45]+features[17][45]+features[18][45]+features[19][45]+features[20][45] + features[21][45] + features[22][45] +features[23][45] +features[24][45]+features[25][45]+features[26][45]+features[27][45]+features[28][45]+features[29][45]+features[30][45] + features[31][45] + features[32][45] +features[33][45] +features[34][45]+features[35][45]+features[36][45]+features[37][45]+features[38][45]+features[39][45]+features[40][45] + features[41][45] + features[42][45] +features[43][45] +features[44][45]+features[45][45]+features[46][45]+features[47][45]+features[48][45]+features[49][45]) / 50

In [ ]:
#將新的特徵全種組合成array

'''

import numpy as np
features_coef = np.array([])
features_coef = np.append(features_coef, F0)
features_coef = np.append(features_coef, F1)
features_coef = np.append(features_coef, F2)
features_coef = np.append(features_coef, F3)
features_coef = np.append(features_coef, F4)
features_coef = np.append(features_coef, F5)
features_coef = np.append(features_coef, F6)
features_coef = np.append(features_coef, F7)
features_coef = np.append(features_coef, F8)
features_coef = np.append(features_coef, F9)
features_coef = np.append(features_coef, F10)
features_coef = np.append(features_coef, F11)
features_coef = np.append(features_coef, F12)
features_coef = np.append(features_coef, F13)
features_coef = np.append(features_coef, F14)
features_coef = np.append(features_coef, F15)
features_coef = np.append(features_coef, F16)
features_coef = np.append(features_coef, F17)
features_coef = np.append(features_coef, F18)
features_coef = np.append(features_coef, F19)
features_coef = np.append(features_coef, F20)
features_coef = np.append(features_coef, F21)
features_coef = np.append(features_coef, F22)
features_coef = np.append(features_coef, F23)
features_coef = np.append(features_coef, F24)
features_coef = np.append(features_coef, F25)
features_coef = np.append(features_coef, F26)
features_coef = np.append(features_coef, F27)
features_coef = np.append(features_coef, F28)
features_coef = np.append(features_coef, F29)
features_coef = np.append(features_coef, F30)
features_coef = np.append(features_coef, F31)
features_coef = np.append(features_coef, F32)
features_coef = np.append(features_coef, F33)
features_coef = np.append(features_coef, F34)
features_coef = np.append(features_coef, F35)
features_coef = np.append(features_coef, F36)
features_coef = np.append(features_coef, F37)
features_coef = np.append(features_coef, F38)
features_coef = np.append(features_coef, F39)
features_coef = np.append(features_coef, F40)
features_coef = np.append(features_coef, F41)
features_coef = np.append(features_coef, F42)
features_coef = np.append(features_coef, F43)
features_coef = np.append(features_coef, F44)
features_coef = np.append(features_coef, F45)

'''

In [ ]:
# LR 總特徵排序

'''
feature_importances = []
ranking = []

feature_importances = pd.DataFrame({"Feature":Z.columns,"Coefficients":features_coef})
ranking = feature_importances.sort_values('Coefficients', ascending=False)
print(ranking)


pd.DataFrame(ranking).to_csv('LR_Frature_Nested.csv')

'''

     Feature  Coefficients
32       crp      0.363158
38      band      0.357333
12   Hbullae      0.205659
8     animal      0.180784
33       seg      0.179092
7        sea      0.172101
28       wbc      0.132671
5       hypo      0.132269
23      alco      0.123350
45    site_U      0.108530
2      fever      0.102261
3      tachy      0.098851
17   Sbullae      0.072327
18      crep      0.066034
15      pain      0.065324
20      hepa      0.057883
24       htn      0.056938
25        lc      0.055015
16     Swell      0.044657
13      anes      0.042447
4      tpnea      0.032133
26        ai      0.027919
11      warm      0.027335
9      farm       0.025580
39  season_F      0.023544
40  season_H      0.023321
37       alt      0.021828
31     sugar      0.021295
0        age      0.017972
22    cancer      0.008808
35        cr      0.004355
19        dm     -0.005262
34        hb     -0.007230
14     necro     -0.009981
42  season_W     -0.016438
1     gender     -0.021308
3